In [36]:
import os, ast, glob
import pandas as pd

In [32]:
def calculate_true_positive(predicted_list, correct_list):
    tp = 0
    for predicted_compound_target in predicted_list:
        if predicted_compound_target in correct_list:   # 'screen' <- predicted 'screen'
            correct_list.remove(predicted_compound_target)
            tp += 1
            continue
        for correct_target in correct_list:
            if predicted_compound_target.find(correct_target) > -1:   # 'audio' <- predicted 'audio aspects'
                correct_list.remove(correct_target)
                tp += 1
                break
    return tp

In [146]:
domain = 'Wireless router'
dfs = []
for pattern in pattern_list:
    filepaths = glob.glob(os.path.join(os.path.dirname(os.getcwd()), 'output', 'preliminary', 'targets', '%s_*_%s.csv' % (domain, pattern)))
    df = pd.read_csv(filepaths[0])
    df['targets'] = df.apply(lambda x: ast.literal_eval(x['targets']), axis=1)
    df['predicted_targets'] = df.apply(lambda x: ast.literal_eval(x['predicted_targets']), axis=1)
    df['tp'] = df.apply(lambda row: calculate_true_positive(row['predicted_targets'], row['targets']), axis=1)
    df['tp'] = df.apply(lambda row: 1 if row['tp']>0 else 0, axis=1)
    pattern_features_df = df[['content', 'tp']]
    pattern_features_df['pattern'] = pattern
    dfs.append(pattern_features_df)
concat_df = pd.concat(dfs, ignore_index=True)    

/home/dmlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [201]:
pattern_evaluation_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'output', 'preliminary', 'sub', '[%s]pattern_evaluation.csv' % domain))
pattern_list = pattern_evaluation_df[pattern_evaluation_df['count']>3].sort_values(by='f1_multiple', ascending=False)['pattern'].values
pattern_list

array(['nsubj', 'amod', 'obl', 'conj-nsubj', 'advmod-nsubj', 'amod-conj',
       'amod-obj', 'amod-obl'], dtype=object)

In [195]:
import copy
whole_patterns_df = pd.DataFrame(concat_df[concat_df['pattern']==concat_df['pattern'].unique()[0]]['content'])
whole_patterns_df['tp'] = whole_patterns_df.apply(lambda x: concat_df[concat_df['content']==x['content']]['tp'].sum(), axis=1)
whole_patterns_df['tp'] = whole_patterns_df.apply(lambda row: 1 if row['tp']>0 else 0, axis=1)

In [203]:
pattern_list[:-2]

array(['nsubj', 'amod', 'obl', 'conj-nsubj', 'advmod-nsubj', 'amod-conj'],
      dtype=object)

nsubj와 가장 redundant한 패턴은 conj-nsubj이다.   
nsubj와 가장 redundant하지 않은 패턴은 amod-obj이다.   
=> amod-obj를 포함시키자

In [213]:
x1 = make_df_including(['nsubj'])['tp'].values.reshape(-1,1)
for pattern2 in pattern_list:
    if pattern2 == 'nsubj': continue
    x2 = make_df_including([pattern2])['tp'].values.reshape(-1,1)
    print('@ %s & %s => %.8f' % (pattern1, pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & amod => 0.00201332


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & obl => 0.00373474


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & amod-conj => 0.00015997


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & amod-obj => -0.00000000
@ nsubj & amod-obl => 0.00005326


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj+amod-obj와 가장 redundant하지 않은 패턴은 amod이다.
=> amod를 포함시키자

In [217]:
patterns1 = ['nsubj', 'amod-obj']
x1 = make_df_including(patterns1)['tp'].values.reshape(-1,1)
for pattern2 in pattern_list:
    if pattern2 in patterns1: continue
    patterns2 = copy.deepcopy(patterns1)
    patterns2.append(pattern2)
    x2 = make_df_including(patterns2)['tp'].values.reshape(-1,1)
    print('@ %s & +%s => %.8f' % (' '.join(patterns1), pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj & +nsubj amod-obj amod => 0.12458932


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj & +nsubj amod-obj obl => 0.16072038


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj & +nsubj amod-obj advmod-nsubj => 0.17297391


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj & +nsubj amod-obj amod-conj => 0.17297391
@ nsubj amod-obj & +nsubj amod-obj amod-obl => 0.18001988


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj+amod-obj+amod와 가장 redundant하지 않은 패턴은 obl이다.   
=> obl를 포함시키자

In [218]:
patterns1 = ['nsubj', 'amod-obj', 'amod']
x1 = make_df_including(patterns1)['tp'].values.reshape(-1,1)
for pattern2 in pattern_list:
    if pattern2 in patterns1: continue
    patterns2 = copy.deepcopy(patterns1)
    patterns2.append(pattern2)
    x2 = make_df_including(patterns2)['tp'].values.reshape(-1,1)
    print('@ %s & +%s => %.8f' % (' '.join(patterns1), pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod & +obl => 0.26718065


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod & +conj-nsubj => 0.28282891


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod & +advmod-nsubj => 0.28282891


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod & +amod-conj => 0.29741646
@ nsubj amod-obj amod & +amod-obl => 0.29741646


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod-obj amod obl와 가장 redundant하지 않은 패턴은 conj-nsubj 와 advmod-nsubj이다.   
=> conj-nsubj를 포함시키자 <= F1 값을 확인해보니 F1값이 줄어든다. 포함시키지 말자. #끝!

In [219]:
patterns1 = ['nsubj', 'amod-obj', 'amod', 'obl']
x1 = make_df_including(patterns1)['tp'].values.reshape(-1,1)
for pattern2 in pattern_list:
    if pattern2 in patterns1: continue
    patterns2 = copy.deepcopy(patterns1)
    patterns2.append(pattern2)
    x2 = make_df_including(patterns2)['tp'].values.reshape(-1,1)
    print('@ %s & +%s => %.8f' % (' '.join(patterns1), pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod obl & +conj-nsubj => 0.30334573


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod obl & +advmod-nsubj => 0.30334573


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod-obj amod obl & +amod-conj => 0.31826528
@ nsubj amod-obj amod obl & +amod-obl => 0.31826528


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Explore

전체 패턴에서 amod-conj, amod-obj, amod-obl은 빠져도 된다. redundant하다.

In [206]:
x1 = make_df_including(pattern_list)['tp'].values.reshape(-1,1)
for pattern2 in pattern_list:
    except_one_pattern_df = make_df(pattern2)
    #x1 = whole_patterns_df['tp'].values.reshape(-1,1)
    x2 = except_one_pattern_df['tp'].values.reshape(-1,1)
    print('@ %s & -%s => %.8f' % ('WHOLE', pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ WHOLE & -nsubj => 0.18868666


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ WHOLE & -amod => 0.16244127


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ WHOLE & -amod-conj => 0.31436199


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ WHOLE & -amod-obj => 0.31436199
@ WHOLE & -amod-obl => 0.31436199


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


우선 amod-obl만 빼보고 다시 확인해보자.
=> 전체 패턴에서 amod-conj, amod-obj은 빠져도 된다. redundant하다.

In [208]:
patterns1 = [item for item in pattern_list if item not in ['amod-obl']]
x1 = make_df_including(patterns1)['tp'].values.reshape(-1,1)
for pattern2 in patterns1:
    patterns2 = [item for item in patterns1 if item not in [pattern2]]
    x2 = make_df_including(patterns2)['tp'].values.reshape(-1,1)
    print('@ %s & -%s => %.8f' % (' '.join(patterns1), pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj amod-conj amod-obj & -nsubj => 0.20208056


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj amod-conj amod-obj & -amod => 0.17192146


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj amod-conj amod-obj & -advmod-nsubj => 0.31079775


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj amod-conj amod-obj & -amod-conj => 0.33327295
@ nsubj amod obl conj-nsubj advmod-nsubj amod-conj amod-obj & -amod-obj => 0.33327295


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


우선 amod-conj, amod-obj만 빼보고 다시 확인해보자.   
=> 전체 패턴에서 conj-nsubj, advmod-nsubj은 빠져도 된다. redundant하다.

In [209]:
patterns1 = [item for item in pattern_list if item not in ['amod-obl', 'amod-conj', 'amod-obj']]
x1 = make_df_including(patterns1)['tp'].values.reshape(-1,1)
for pattern2 in patterns1:
    patterns2 = [item for item in patterns1 if item not in [pattern2]]
    x2 = make_df_including(patterns2)['tp'].values.reshape(-1,1)
    print('@ %s & -%s => %.8f' % (' '.join(patterns1), pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj & -nsubj => 0.20208056


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj & -amod => 0.16117380


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj & -obl => 0.28225733


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl conj-nsubj advmod-nsubj & -conj-nsubj => 0.31079775
@ nsubj amod obl conj-nsubj advmod-nsubj & -advmod-nsubj => 0.31079775


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


우선 conj-nsubj, advmod-nsubj만 빼보고 다시 확인해보자.   
=> 전체 패턴에서 conj-nsubj, advmod-nsubj은 빠져도 된다. redundant하다.

In [211]:
patterns1 = [item for item in pattern_list if item not in ['amod-obl', 'amod-conj', 'amod-obj', 'advmod-nsubj', 'conj-nsubj']]
x1 = make_df_including(patterns1)['tp'].values.reshape(-1,1)
for pattern2 in patterns1:
    patterns2 = [item for item in patterns1 if item not in [pattern2]]
    x2 = make_df_including(patterns2)['tp'].values.reshape(-1,1)
    print('@ %s & -%s => %.8f' % (' '.join(patterns1), pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl & -nsubj => 0.14585260


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj amod obl & -amod => 0.14585260
@ nsubj amod obl & -obl => 0.26718065


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


amod는 nsubj와 가장 독립적인 feature임

In [197]:
def sum_tps(content, except_one_pattern):
    df = concat_df[concat_df['pattern']!=except_one_pattern]
    return df[df['content']==content]['tp'].sum()
    
def make_df(except_one_pattern):
    df = pd.DataFrame(concat_df[concat_df['pattern']==concat_df['pattern'].unique()[0]]['content'])
    df['tp'] = df.apply(lambda x: sum_tps(x['content'], except_one_pattern), axis=1)
    df['tp'] = df.apply(lambda row: 1 if row['tp']>0 else 0, axis=1)
    return df.sort_values(by='content')

def sum_tps_including(content, include_patterns):
    df = concat_df[concat_df['pattern'].isin(include_patterns)]
    return df[df['content']==content]['tp'].sum()

def make_df_including(include_patterns):
    df = pd.DataFrame(concat_df[concat_df['pattern']==concat_df['pattern'].unique()[0]]['content'])
    df['tp'] = df.apply(lambda x: sum_tps_including(x['content'], include_patterns), axis=1)
    df['tp'] = df.apply(lambda row: 1 if row['tp']>0 else 0, axis=1)
    return df.sort_values(by='content')

current_idx = 0
current_pattern = pattern_list[current_idx]
next_five = pattern_list[current_idx+1:current_idx+6]
whole_patterns_df = whole_patterns_df.sort_values(by='content')

pattern1 = 'nsubj'
x1 = make_df_including([pattern1])['tp'].values.reshape(-1,1)
for pattern2 in pattern_list:
    if pattern1 == pattern2: continue
    #except_one_pattern_df = make_df(pattern2)
    #x1 = whole_patterns_df['tp'].values.reshape(-1,1)
    #x2 = except_one_pattern_df['tp'].values.reshape(-1,1)
    x2 = make_df_including([pattern1, pattern2])['tp'].values.reshape(-1,1)
    print('@ %s & +%s => %.4f' % (pattern1, pattern2, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & +amod => 0.1246


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & +obl => 0.1607


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & +amod-conj => 0.1730


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


@ nsubj & +amod-obj => 0.1854
@ nsubj & +amod-obl => 0.1800


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


obl은 nsubj+amod와 가장 독립적인 feature임

In [198]:
p1, p2 = 'nsubj', 'amod'
x1 = make_df_including([p1, p2])['tp'].values.reshape(-1,1)
for p3 in pattern_list:
    if p3 in [p1, p2]: continue
    #except_one_pattern_df = make_df(pattern2)
    #x1 = whole_patterns_df['tp'].values.reshape(-1,1)
    #x2 = except_one_pattern_df['tp'].values.reshape(-1,1)
    x2 = make_df_including([p1, p2, p3])['tp'].values.reshape(-1,1)
    print('%s & +%s => %.4f' % (' '.join([p1,p2]), p3, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod & +obl => 0.2672


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod & +conj-nsubj => 0.2828


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod & +amod-conj => 0.2974


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod & +amod-obj => 0.2974
nsubj amod & +amod-obl => 0.2974


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


conj-nsubj은 nsubj+amod+obl와 가장 독립적인 feature임

In [199]:
p1, p2, p3 = 'nsubj', 'amod', 'obl'
x1 = make_df_including([p1, p2, p3])['tp'].values.reshape(-1,1)
for p4 in pattern_list:
    if p4 in [p1, p2, p3]: continue
    #except_one_pattern_df = make_df(pattern2)
    #x1 = whole_patterns_df['tp'].values.reshape(-1,1)
    #x2 = except_one_pattern_df['tp'].values.reshape(-1,1)
    x2 = make_df_including([p1, p2, p3, p4])['tp'].values.reshape(-1,1)
    print('%s & +%s => %.4f' % (' '.join([p1,p2, p3]), p4, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl & +conj-nsubj => 0.3033


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl & +advmod-nsubj => 0.3033


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl & +amod-conj => 0.3183


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl & +amod-obj => 0.3183
nsubj amod obl & +amod-obl => 0.3183


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [200]:
x1 = make_df_including(pattern_list[:4])['tp'].values.reshape(-1,1)
for p5 in pattern_list:
    if p5 in pattern_list[:4]: continue
    #except_one_pattern_df = make_df(pattern2)
    #x1 = whole_patterns_df['tp'].values.reshape(-1,1)
    #x2 = except_one_pattern_df['tp'].values.reshape(-1,1)
    x2 = make_df_including(pattern_list[:4])['tp'].values.reshape(-1,1)
    print('%s & +%s => %.4f' % (' '.join(pattern_list[:4]), p5, mutual_info_classif(x1, x2, discrete_features=[0])))

/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl conj-nsubj & +advmod-nsubj => 0.3258


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl conj-nsubj & +amod-conj => 0.3258


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


nsubj amod obl conj-nsubj & +amod-obj => 0.3258
nsubj amod obl conj-nsubj & +amod-obl => 0.3258


/home/dmlab/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
